# constraints

Este notebook se encarga de asignar las llaves primarias y foráneas a las tablas de hechos y dimensiones

### Importación de librerías

In [ ]:
import yaml
from sqlalchemy import create_engine, text
from sqlalchemy.inspection import inspect

### Definición de las llaves foráneas

In [ ]:
props = {
    "fact_novelty" : {
        "key_type_novelty" : {
            "column" : "key_type_novelty",
            "ref_table" : "dim_type_novelty",
            "ref_column" : "key_dim_type_novelty"
        },
    },
    "fact_messaging_daily" : {
        "key_date" : {
            "column" : "key_date",
            "ref_table" : "dim_date",
            "ref_column" : "key_dim_date"
        },
        "key_client" : {
            "column" : "key_client",
            "ref_table" : "dim_client",
            "ref_column" : "key_dim_client"
        },
        "key_headquarter" : {
            "column" : "key_headquarter",
            "ref_table" : "dim_headquarter",
            "ref_column" : "key_dim_headquarter"
        },
    },
    "fact_messaging_time" : {
        "key_time" : {
            "column" : "key_time",
            "ref_table" : "dim_time",
            "ref_column" : "key_dim_time"
        },
        "key_messenger" : {
            "column" : "key_messenger",
            "ref_table" : "dim_messenger",
            "ref_column" : "key_dim_messenger"
        },
    },
    "fact_messaging_accumulating" : {
        "key_start_date" : {
            "column" : "key_start_date",
            "ref_table" : "dim_date",
            "ref_column" : "key_dim_date"
        },
        "key_assignment_date" : {
            "column" : "key_assignment_date",
            "ref_table" : "dim_date",
            "ref_column" : "key_dim_date"
        },
        "key_pick_up_date" : {
            "column" : "key_pick_up_date",
            "ref_table" : "dim_date",
            "ref_column" : "key_dim_date"
        },
        "key_delivery_date" : {
            "column" : "key_delivery_date",
            "ref_table" : "dim_date",
            "ref_column" : "key_dim_date"
        },
        "key_closing_date" : {
            "column" : "key_closing_date",
            "ref_table" : "dim_date",
            "ref_column" : "key_dim_date"
        },
        "key_start_time" : {
            "column" : "key_start_time",
            "ref_table" : "dim_time",
            "ref_column" : "key_dim_time"
        },
        "key_assignment_time" : {
            "column" : "key_assignment_time",
            "ref_table" : "dim_time",
            "ref_column" : "key_dim_time"
        },
        "key_pick_up_time" : {
            "column" : "key_pick_up_time",
            "ref_table" : "dim_time",
            "ref_column" : "key_dim_time"
        },
        "key_delivery_time" : {
            "column" : "key_delivery_time",
            "ref_table" : "dim_time",
            "ref_column" : "key_dim_time"
        },
        "key_closing_time" : {
            "column" : "key_closing_time",
            "ref_table" : "dim_time",
            "ref_column" : "key_dim_time"
        },
    },
}

### Definición de funciones

Estas funciones se encargan de verificar y asignar las llaves primarias y foráneas

In [ ]:
def define_primary_key(conn, table_name, column_name):
    try:
        query = text(f'ALTER TABLE {table_name} ADD PRIMARY KEY ({column_name})')
        conn.execute(query)
        conn.commit()
        print(f"Primary key added on {table_name} for column {column_name}.")
    except Exception as e:
        print(f"Error adding primary key on {table_name}: {e}")

def primary_key_exists(inspector, table_name):
    pk_constraint = inspector.get_pk_constraint(table_name)
    return pk_constraint is not None and len(pk_constraint.get('constrained_columns', [])) > 0

def define_foreigns_key(conn, table_name, column_name, ref_table, ref_column):
    try:
        query = text(f'ALTER TABLE {table_name} ADD FOREIGN KEY ({column_name}) REFERENCES {ref_table}({ref_column})')
        conn.execute(query)
        conn.commit()
        print(f"Foreign key added on {table_name} for column {column_name}.")
    except Exception as e:
        print(f"Error adding foreign key on {table_name}: {e}")

def foreign_key_exists(inspector, table_name, column_name):
    fk_constraints = inspector.get_foreign_keys(table_name)
    for fk in fk_constraints:
        if column_name in fk.get('constrained_columns', []):
            return True
    return False

### Asignación de llaves

In [ ]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    configSource = config['source']
    configDestination = config['destination']

urlDestination = f"{configDestination['driver']}://{configDestination['user']}:{configDestination['password']}@{configDestination['host']}:{configDestination['port']}/{configDestination['db']}"

engineDestination = create_engine(urlDestination)

inspector = inspect(engineDestination)
tnames = inspector.get_table_names()

with engineDestination.connect() as conn: 
    for table in tnames:
        primary_key_column = f'key_{table}'
        
        columns = inspector.get_columns(table)
        column_names = [column['name'] for column in columns]

        if primary_key_exists(inspector, table):
            print(f"Table {table} already has a primary key. Skipping.")
            continue

        if primary_key_column in column_names:
            define_primary_key(conn, table, primary_key_column)
        else:
            print(f"Column {primary_key_column} does not exist in table {table}.")

    conn.close()

print("\n")

with engineDestination.connect() as conn:
    for fact_table, foreign_keys in props.items():
        print("fact_table>", fact_table)
        for foreign_key, foreign_keys_info in foreign_keys.items():
            print(">")
            print("foreign_key>", foreign_key)

            if not foreign_key_exists(inspector, fact_table, foreign_keys_info['column']):
                define_foreigns_key(conn, fact_table, foreign_keys_info['column'], foreign_keys_info['ref_table'], foreign_keys_info['ref_column'])
            else:
                print(f"Foreign key already exists on {table} for column {foreign_keys_info['column']}. Skipping.")
        print("=================")

            
    conn.close